# 02 DATA ACQUISITION AND UDERSTANDING

## Goals

Produce a clean, high-quality data set whose relationship to the target variables is understood. Locate the data set in the appropriate analytics environment so you are ready to model.
Develop a solution architecture of the data pipeline that refreshes and scores the data regularly.

## How to do it

There are three main tasks addressed in this stage:

0. Ingest the data into the target analytic environment.
0. Explore the data to determine if the data quality is adequate to answer the question.
0. Set up a data pipeline to score new or regularly refreshed data.

Our source: https://ourworldindata.org/covid-cases

# 0.0. Imports

In [ ]:
# data manipulation
import pandas as pd
import numpy as np
import datetime as dt

# warning
import warnings
warnings.filterwarnings('ignore')

# graphs
import matplotlib.pyplot as plt
import seaborn as sns
from darkstyle import dark_style as dks

# statistic
from scipy import stats


# scalers
from sklearn.preprocessing import MinMaxScaler, RobustScaler, OneHotEncoder, LabelEncoder

# feature selection
from boruta import BorutaPy

# model selection
from sklearn.model_selection import train_test_split

# machine learning models
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor

# model's cross-validation
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import cross_validate, cross_val_score, cross_val_predict, HalvingGridSearchCV

# model's metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# save files
import pickle

## 0.1. Helper Functions

In [ ]:
# set some default figure paramenters and style
def settings():
    dks.dark_style() # module for matplot darkstyle
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 8
    pd.options.display.max_columns = None
    pd.set_option( 'display.expand_frame_repr', False )
settings()

# cramers v statistic function for categorical values
def cramers_corrected_stat(x, y):
    """ calculate Cramers V statistic for categorial-categorial association.
    """
    # Calculate confusion matrix
    cm = pd.crosstab(x, y).values
    n = cm.sum()
    r, k = cm.shape

    # Calculate chi2
    chi2 = stats.chi2_contingency(cm)[0]
    # Calculate chi2 correction
    chi2corr = max(0, chi2 - (k-1)*(r-1)/(n-1))
    # K correction
    kcorr = k - (k-1)**2/(n-1)
    # R correction
    rcorr = r - (r-1)**2/(n-1)
    
    return np.sqrt((chi2corr/n) / (min(kcorr-1, rcorr-1)))

# model's performance function
def error(model, y_test, y_pred):
    root_mse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    rs = r2_score(y_test, y_pred)
    
    return pd.DataFrame( {
        'Model' : model,
        'MAE' : mae,
        'R2' : rs,
        'RMSE' : root_mse
           }, index=[0])

# cross Validation Function
def cross_validation(model_name, model, x, y):

    # Error lists to concatenate the values
    rmse_list = cross_val_score(model, x, y, scoring='neg_root_mean_squared_error', cv=5)
    mae_list = cross_val_score(model, x, y, scoring='neg_mean_absolute_error', cv=5)
    r2_list = cross_val_score(model, x, y, scoring='r2', cv=5)

    return pd.DataFrame( {
        'Model Name' : model_name,
        'MAE' : np.round(np.mean(-mae_list), 4),
        'MAE STD' : np.round(np.std(mae_list), 4),
        'R2' : np.mean(np.mean(r2_list)),
        'RSME' : np.round(np.mean(-rmse_list), 4),
        'RSME STD' : np.round(np.std(rmse_list), 4)
    }, index=[0]) 

# 1.0. Data

## 1.1. Load the Data

In [ ]:
# loading main file
url = 'https://covid.ourworldindata.org/data/owid-covid-data.csv'
df = pd.read_csv(url)

# load describe columns file
cols_describe = pd.read_csv('dataset/describe.csv')

In [ ]:
df.head(2)

## 1.2. Knowing the Data

### 1.2.1. Data shape

In [ ]:
print(f'The dataset shape is: {df.shape}')

### 1.2.2. Type and Structure

In [ ]:
df.info(show_counts=True)

* Some features has Null values;
* dtypes: float64(54), object(5);
* Total entries: 75558;
* Total of 59 columns;

### 1.2.3. Checking Missing Values

In [ ]:
df.isnull().sum()

Some features has many null values, the drop technique was not applied because the missing values are due to events that have not occurred yet.

In [ ]:
# checking miss values rate
missing_values = []
for col in df.columns:
    total = len(df[col])
    total_missing = df[col].isna().sum()
    missing_rate = total_missing/total
    # append to list
    if missing_rate > 0.6:
        missing_values.append(col)

missing_values

In [ ]:
# drop columns with too much missing values
#df2 = df.drop(columns = columns_to_drop, axis=1)

## 1.6. Descriptive Statistics

### 1.6.1. Numerical Attributes

In [ ]:
# selecting numerical attributes
num_df = df.select_dtypes(include=['float64'])

# describe
describe = num_df.describe().T

# adding other metrics to knowing data
describe['range'] = (num_df.max() - num_df.min()).tolist()
describe['unique val.'] = num_df.nunique()
describe['variation coefficient'] = np.round((num_df.std() / num_df.mean()), 4).tolist()
describe['skew'] = np.round(num_df.skew(), 4).tolist()
describe['kurtosis'] = np.round(num_df.kurtosis(), 4).tolist()

describe

In [ ]:
# about skewness
high_skewness = []
for feat in range(len(describe.index)):
    if abs(describe['skew'].iloc[feat]) > 2:
        high_skewness.append(describe.index[feat])
print(f'There\'s {len(high_skewness)} features with high skew:')
print(high_skewness)

In [ ]:
# about kurstosis
high_kurstosis = []
for feat in range(len(describe.index)):
    if abs(describe['kurtosis'].iloc[feat]) > 3:
        high_kurstosis.append(describe.index[feat])
print(f'There\'s {len(high_kurstosis)} features with high kurtosis:')
print(high_kurstosis)

In [ ]:
# negative values
negative_values = []
for feat in range(len(describe.index)):
    if abs(describe['min'].iloc[feat] <= 0):
        negative_values.append(describe.index[feat])
print(f'There\'s {len(negative_values)} features with negative values:')
print(negative_values)

The negative values have no meaning to events. It was, probably, a typo. 
Negative values often seem very distant from behavior, let's replace the values with 0.
The skewness and high kurtosis values, we will dealing with those features later.

In [ ]:
# replace negative values with mean
for i in negative_values:
    df[i] = df[i].apply(lambda x : x if x > 0 else 0)

### 1.6.2. Categorical Attributes

In [ ]:
cat_df = df.select_dtypes(exclude='float64')

cat_df.describe().T

* There is 215 countries in the present dataset;
* 6 continents;
* continents have null values;

# 2.0. Feature Engineering and Hypothesis Creation

## 2.1. Feature Engineering

In [ ]:
# transform object in datetime format
df['date'] = pd.to_datetime(df.date)

# creating year column
df['year'] = df.date.dt.year

# creating intervals
df['year_month'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m')

In [ ]:
# sorting by year and month
df = df.sort_values('year_month').reset_index(drop=True)

# removing wordwild entries
df = df[~df['iso_code'].str.contains('OWID')]

## 2.3. Hypothesis

#### Hypothesis summary

| INDEX | HYPOTHESIS                                                   |
| ----- | ------------------------------------------------------------ |
| H1    | Brazil more likely to Covid19 than USA in total cases.       |
| H2    | Continent's with high vaccination rate are more effective on Covid19 control. |
| H3    | Countries with high populatation density are more likely to Covid19. |
| H4    | Countries with high elderly are most affected by Covid19.    |
| H5    | Countries with high GDP are less likely to covid19.          |
| H6    | USA has more Covid19 death.                                  |
| H7    | European continent conducted the highest number of tests.    |
| H8    | Brazil is more likely to new cases of Covid19.               |
| H9    | USA is more likely to vaccination.                           |
| H10   | Brazil is more likely Covid19 in South America.              |

# 3.0. Exploratory Data Analysis

## 3.1. Univariate Analysis

### 3.1.1. Target Variable

Our target variable is total_cases

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=1)

# plot the density   
sns.histplot(df.total_cases, kde=True, ax=ax[0])
ax[0].set_ylabel('Frequency')

# Plot the boxplot   
sns.boxplot(df.total_cases, ax=ax[1])
ax[1].set_xlabel('Value')

# plot the lineplot
sns.lineplot(data=df, x='year_month', y='total_cases', ax=ax[2])
ax[2].set_title('Trend')

# Add a title to the Figure
fig.suptitle('Data Distribution')

# savefig
#plt.savefig('img/fig00a')

# Show the figure
plt.show()

In [ ]:
 plot the density   
sns.histplot(np.log(df.total_cases), kde=True)
ax[0].set_ylabel('Frequency')

# savefig
#plt.savefig('img/fig00b')

# Show the figure
plt.show()

### 3.1.2. Numerical variables

In [ ]:
# selecting numerical with filtering values
num_df2 = df.select_dtypes(include='float64')

# removing total_cases column
numerical = num_df2.drop('total_cases', axis=1)

# plot 
numerical.hist(bins=25)

plt.tight_layout()

#plt.savefig('img/fig01')

plt.show()

In [ ]:
# removing total_cases 
high_skewness.remove('total_cases')

# selecting columns with high skew
numerical_selected = numerical[high_skewness]

# removing negative values
no_neg = numerical_selected[numerical_selected >= 0]

# plot
np.log1p(numerical_selected).hist(bins=25)
plt.tight_layout()
#plt.savefig('img/fig02')
plt.show()

### 3.1.3. Checking outliers

In [ ]:
# sets the initial plot position
n = 1

# iterate over the columns to plot
for column in numerical.columns:
    plt.subplot(8, 7, n)
    _ = sns.boxplot(numerical[column])
    n += 1
    
# adjusts vertical space between plots
plt.subplots_adjust(hspace=0.5)

#plt.savefig('img/fig03')

# displays the plot
plt.show()

In [ ]:
# grouping features with high kurtosis
high_outliers = list(set(high_kurstosis)| set(high_skewness))
print('Columns with high outliers influence:')
print(high_outliers)

### 3.1.3. Categorical variable

In [ ]:
# subplot 1
plt.subplot(2,2,1)
sns.countplot(cat_df.iso_code)

# subplot 2
plt.subplot(2,2,2)
sns.countplot(cat_df.continent)

# subplot 3
plt.subplot(2,2,3)
sns.countplot(cat_df.location)

# subplot 4
plt.subplot(2,2,4)
sns.countplot(cat_df.tests_units)

# save plot
#plt.savefig('img/fig04')

# plot
plt.show()

In [ ]:
# saving optimize dataset
df.to_csv('dataset/df1.csv', index=False)

In [ ]:
# checkpoint 1
df2 = pd.read_csv('dataset/df1.csv')

## 3.2. Bivariate Analysis

### H1. Brazil more likely to Covid19 than USA in total cases. (TRUE)
Brazil has a smaller population and has more cases than USA.

In [ ]:
# selecting features
auxh1 = df2[['iso_code', 'population', 'year_month', 'total_cases', 'total_cases_per_million']]

# selecting countries
bra_usa =  auxh1.query('(iso_code == "BRA") | (iso_code == "USA")').sort_values('year_month')

fig, ax = plt.subplots(nrows=1, ncols=3)
# subplot 1
sns.barplot(data=bra_usa, x='iso_code', y='population', ax= ax[0])
ax[0].set_title('USA X BRA POPULATION')

# subplot 2
sns.lineplot(data=bra_usa, x='year_month', y='total_cases', hue='iso_code', ci=None, ax=ax[1])
ax[1].tick_params(rotation=45)
ax[1].set_title('TOTAL CASES')

# subplot 3
sns.lineplot(data=bra_usa, x='year_month', y='total_cases_per_million', hue='iso_code', ci=None, ax=ax[2])
ax[2].tick_params(rotation=45)
ax[2].set_title('TOTAL CASES PER MILLION')

# save plot
plt.savefig('img/fig05')

# plot
plt.show()

### H2. Continentes with high vaccination rate are more effective on Covid19 control. (PERHAPS)
There are still not enough elements since the vaccine is a recent event, but the continents with the highest numbers of cases are vaccinating more.
A strong correlation between total cases and total vaccine.

In [ ]:
auxh2 = df2[['continent', 'year_month', 'date', 'total_cases', 'total_vaccinations',
       'total_cases_per_million', 'total_vaccinations_per_hundred']]

# subplot 1
plt.subplot(2,3,1)
sns.barplot(data=auxh2, x='continent', y='total_cases')
plt.title('CONTINENTS TOTAL CASE')

# subplot 2
plt.subplot(2,3,2)
sns.lineplot(data=auxh2, hue='continent', x='year_month', y='total_vaccinations', ci=None)
plt.xticks(rotation=90)
plt.title('TOTAL VACCINATION')

# subplot 3
plt.subplot(2,3,3)
sns.heatmap(auxh2[['total_cases', 'total_vaccinations']].corr(method='pearson'), annot=True)

# subplot 4
plt.subplot(2,3,4)
sns.barplot(data=auxh2, x='continent', y='total_cases_per_million')
plt.title('CONTINENTS TOTAL CASE PER MILLION')

# subplot 5
plt.subplot(2,3,5)
sns.lineplot(data=auxh2, hue='continent', x='year_month', y='total_vaccinations_per_hundred', ci=None)
plt.xticks(rotation=90)
plt.title('TOTAL VACCINATIONS PER HUNDRED')

# subplot 6
plt.subplot(2,3,6)
sns.heatmap(auxh2[['total_cases', 'total_vaccinations_per_hundred']].corr(method='pearson'), annot=True)

# adjust
plt.tight_layout()

# save plot
plt.savefig('img/fig06')

# plot
plt.show()

### H3. Countries with high populatation density are more likely to Covid19. (FALSE)
There is a weak correlation between the total number of cases and population density. In top 5 countries with higher cases, only Indian has high population density.

In [ ]:
# selecting columns
auxh3 = df2[['iso_code', 'total_cases', 'population_density']]

# removing worldwide variable and groupby countries
auxh3_gp = auxh3.groupby('iso_code').sum().reset_index()

# sorting values
auxh3_gp = auxh3_gp.sort_values('total_cases', ascending=False)

# initialize figure
fig, ax = plt.subplots(nrows=3, ncols=1)

# load total_cases
sns.barplot(data=auxh3_gp.head(50), x='iso_code', y='total_cases', color='w', ax=ax[0])
ax[0].set_title('COVID 19 HIGHEST CASE TOP 50')

# plot the population density
sns.barplot(data = auxh3_gp.head(50), x='iso_code', y='population_density', color='y', ax=ax[1])
ax[1].invert_yaxis()
ax[1].set_title('POPULATION DENSITY')

# heatmap
sns.heatmap(auxh3[['total_cases', 'population_density']].corr(method='pearson'), annot=True, ax=ax[2])

# adjust
plt.tight_layout()

# save plot
plt.savefig('img/fig07')

# plot
plt.show()

### H4. Countries with high elderly are most affected by Covid19. (FALSE)
Most countries with high rate of elderly people have low rate of total cases

In [ ]:
auxh4 = df2[['iso_code', 'total_cases', 'aged_65_older', 'aged_70_older']]

auxh4['elderly'] = auxh4['aged_65_older'] + auxh4['aged_70_older']

auxh4_gp = auxh4.groupby('iso_code').mean().reset_index()

# sorting values
auxh4_gp = auxh4_gp.sort_values('total_cases', ascending=False)

# set fig
fig, ax = plt.subplots(nrows=3, ncols=1)

# ax 1
sns.barplot(data=auxh4_gp.head(50), x='iso_code', y='elderly',\
    color='cyan', ax=ax[0])
ax[1].set_title('ELDERLY RATE')

# ax 2
sns.barplot(data=auxh4_gp.head(50), x='iso_code', y='total_cases',\
    color='white', ax=ax[1])
ax[1].invert_yaxis()
ax[1].set_title('TOTAL CASES')

# ax 3
sns.heatmap(auxh4[['total_cases', 'elderly']].corr(method='pearson'), annot=True, ax=ax[2])

# adjust
plt.tight_layout()

# save plot
plt.savefig('img/fig08')

# plot
plt.show()

### H5. Countries with high GDP are less likely to covid19. (FALSE)
Countries are being affected regardless of their GDP

In [ ]:
auxh5 = df2[['iso_code', 'total_cases', 'gdp_per_capita', 'extreme_poverty']]

auxh5_gp = auxh5[~auxh5.iso_code.str.contains('OWID')].groupby('iso_code').sum().reset_index()

# sorting values
auxh5_gp = auxh5_gp.sort_values('total_cases', ascending=False)

# subplot 1
plt.subplot(2,2,1)
sns.barplot(data=auxh5_gp.head(30), x='iso_code', y='gdp_per_capita')
plt.title('GDP PER COUNTRY')

# subplot 2
plt.subplot(2,2,2)
sns.regplot(data=auxh5_gp, x='total_cases', y='extreme_poverty')

# subplot 3
plt.subplot(2,2,3)
sns.barplot(data=auxh5_gp.head(30), x='iso_code', y='extreme_poverty')
plt.title('EXTREME POVERTY')

# subplot 4
plt.subplot(2,2,4)
sns.heatmap(auxh5[['total_cases', 'gdp_per_capita', 'extreme_poverty']].corr(method='pearson'), annot=True)

plt.tight_layout()

# save plot
plt.savefig('img/fig09')

plt.show()

### H6. USA has more Covid19 death. (TRUE)
USA leads total death toll per covid and has the most deaths per million in north america.

In [ ]:
auxh6 = df2[['iso_code', 'continent', 'total_deaths', 'total_cases', 'total_deaths_per_million', 'year_month']]

auxh6_gp = auxh6[~auxh6.iso_code.str.contains('OWID')].groupby('iso_code').sum().reset_index()

auxh6_gp = auxh6_gp.sort_values('total_deaths', ascending=False)

# fig ax
fig, ax = plt.subplots(nrows = 2, ncols = 1)

# plot 1
sns.barplot(data=auxh6_gp.head(50), y='total_deaths', x='iso_code', ax=ax[0])

# plot 2
#sns.barplot(data=auxh6_gp.head(50), y='total_cases', x='iso_code', ax=ax[1])
#ax[1].invert_yaxis()

# plot 3
north_america = df2.query('continent == "North America"')
sns.lineplot(data=north_america, x='year_month', y='total_deaths_per_million',\
     hue='iso_code', style='iso_code', ci=None, ax=ax[1])

plt.tight_layout()

# save plot
plt.savefig('img/fig10')

plt.show()

### H7. European continent conducted the highest number of tests. (TRUE)
USA has the highest total vaccination, but Europe vaccinated more proportionately.

In [ ]:
auxh7 = df2[['continent', 'total_tests', 'total_tests_per_thousand', 'year_month']]

fig, ax = plt.subplots(nrows=2, ncols=1)

# ax 1
sns.barplot(data=auxh7.sort_values('total_tests', ascending=False),\
     x='continent', y='total_tests', ax=ax[0])
ax[0].set_title('CONTINENT - TOTAL TESTS')

# ax 2
sns.lineplot(data=auxh7, x='year_month', y='total_tests_per_thousand',\
    hue='continent', ax=ax[1])
ax[1].set_title('CONTINENT - TOTAL TESTS PER THOUSAND')

# save plot
plt.savefig('img/fig11')

# plot
plt.show()

### H8. Brazil is more likely to new cases of Covid19. (FALSE)
Brazil is the third in number of total cases.


In [ ]:
auxh8 = df2[['iso_code', 'year_month', 'new_cases', 'new_cases_per_million']]

auxh8_gp = auxh8.groupby(['iso_code', 'year_month']).sum().reset_index()

# sorting values
auxh8_gp1 = auxh8_gp.sort_values('new_cases', ascending=False)
auxh8_gp2 = auxh8_gp.sort_values('new_cases_per_million', ascending=False)

# subplot 1
plt.subplot(2,2,1)
sns.barplot(data=auxh8_gp1.head(50), x='iso_code', y='new_cases')
plt.title('NEW CASES')

# subplot 2
plt.subplot(2,2,3)
sns.barplot(data=auxh8_gp2.head(50), x='iso_code', y='new_cases_per_million')
plt.title('NEW CASES PER MILLION')

# subplot 3
plt.subplot(2,2,2)
sns.regplot(data=auxh8, x='new_cases_per_million', y='new_cases', ci=None)

# subplot 4
plt.subplot(2,2,4)
sns.heatmap(auxh8[['new_cases', 'new_cases_per_million']].corr(method='pearson'), annot=True)

# adjust
plt.tight_layout()

# save plot
plt.savefig('img/fig12')

# show
plt.show()

### H9. USA is more likely to vaccination. (PERHAPS)
USA has the most total vaccination, but not in proportion, large part of the population is still waiting for the vaccine.

In [ ]:
auxh9 = df2[['iso_code', 'year_month', 'total_vaccinations', 
            'total_vaccinations_per_hundred']]

auxh9_gp = auxh9[~auxh9.iso_code.str.contains('OWID')].groupby(['iso_code', 'year_month']).sum().reset_index()

# sorting values
auxh9_gp1 = auxh9_gp.sort_values('total_vaccinations', ascending=False)
auxh9_gp2 = auxh9_gp.sort_values('total_vaccinations_per_hundred', ascending=False)
auxh9_gp3 = auxh9_gp.sort_values('year_month', ascending=False)

# subplot 1
plt.subplot(2,2,1)
sns.barplot(data=auxh9_gp1.head(50), x='iso_code', y='total_vaccinations')
plt.title('COUNTRIES - TOTAL VACCINATIONS')

# subplot 2
plt.subplot(2,2,2)
sns.barplot(data=auxh9_gp2.head(50), x='iso_code', y='total_vaccinations_per_hundred')
plt.title('COUNTRIES - TOTAL VACCINATIONS PER HUNDRED')

# subplot 3
plt.subplot(2,2,3)
sns.lineplot(data=auxh9, x='year_month', y='total_vaccinations', ci=None)
plt.title('TOTAL VACCINATIONS')

# subplot 4
plt.subplot(2,2,4)
sns.heatmap(auxh9[['total_vaccinations', 'total_vaccinations_per_hundred']].corr(method='pearson'), annot=True)

# adjust
plt.tight_layout()

# save plot
plt.savefig('img/fig13')

# show
plt.show()

### H10. Brasil is more likely Covid19 in South America. (TRUE)
In south america Brazil has the highest number of cases.

In [ ]:
auxh10 = df2[['iso_code', 'continent', 'year_month', 'total_cases', 
            'total_cases_per_million']]

south_america = auxh10.query('continent == "South America"')

# subplot 1
plt.subplot(3,1,1)
sns.lineplot(data=south_america, x='year_month', y='total_cases', hue='iso_code',\
ci=None)
plt.title('SOUTH AMERICA - TOTAL CASES')

# subplot 2
plt.subplot(3,1,2)
sns.lineplot(data=south_america, x='year_month', y='total_cases_per_million',\
hue='iso_code', ci=None)
plt.title('SOUTH AMERICA - TOTAL VACCINATIONS PER MILLION')

# subplot 3
plt.subplot(3,1,3)
sns.heatmap(auxh10[['total_cases', 'total_cases_per_million']].corr(method='pearson'), annot=True)

# adjust
plt.tight_layout()

# save plot
plt.savefig('img/fig14')

# show
plt.show()

In [ ]:
path = gpd.datasets.get_path('naturalearth_lowres')
world = gpd.read_file(path)

world

## 3.3. Multivariate Analysis

### 3.3.1. Numerical variables

In [ ]:
num_df = df2.select_dtypes(include='float64')

sns.heatmap(num_df.corr(), annot=True, cmap='YlGnBu')

# save plot
plt.savefig('img/fig15')

plt.show()

In [ ]:
# high correlation with target variable
columns = ['new_cases','new_cases_smoothed', 'total_deaths', 'new_deaths',
       'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million']

### 3.3.2. Categorical variables

In [ ]:
cat_df = df2.select_dtypes(exclude='float64')

dict_for_corr = {}

columns = cat_df.columns.tolist()

for column in columns:
    dict_for_corr[column] = {}

    for column2 in columns:
        dict_for_corr[column][column2] = cramers_corrected_stat(cat_df[column], cat_df[column2])

corr_cat = pd.DataFrame(dict_for_corr)

sns.heatmap(corr_cat, annot=True)

# save plot
plt.savefig('img/fig16')

plt.show()

# 03. Modeling stage of the Team Data Science Process lifecycle

## Goals

* Determine the optimal data features for the machine-learning model.
* Create an informative machine-learning model that predicts the target most accurately.
* Create a machine-learning model that's suitable for production.


## Model training

Depending on the type of question that you're trying to answer, there are many modeling algorithms available. For guidance on choosing the algorithms, see How to choose algorithms for Microsoft Azure Machine Learning. Although this article uses Azure Machine Learning, the guidance it provides is useful for any machine-learning projects.

The process for model training includes the following steps:

* Split the input data randomly for modeling into a training data set and a test data set.
* Build the models by using the training data set.
* Evaluate the training and the test data set. Use a series of competing machine-learning algorithms along with the various associated tuning parameters (known as a parameter sweep) that are geared toward answering the question of interest with the current data.
* Determine the “best” solution to answer the question by comparing the success metrics between alternative methods.

## Artifacts

The artifacts produced in this stage include:

* Feature sets: The features developed for the modeling are described in the Feature sets section of the Data definition report. It contains pointers to the code to generate the features and a description of how the feature was generated.
* Model report: For each model that's tried, a standard, template-based report that provides details on each experiment is produced.
* Checkpoint decision: Evaluate whether the model performs sufficiently for production. Some key questions to ask are:
    * Does the model answer the question with sufficient confidence given the test data?
    * Should you try any alternative approaches? Should you collect additional data, do more feature engineering, or experiment with other algorithms?




# 4.0. Data Preparation

In [ ]:
# loading dataset for preparation
df3 = pd.read_csv('dataset/df1.csv')

We are trying to predict Covid19 total cases, so we will drop the columns related to the number of contaminated and killed by Covid19.

In [ ]:
# columns with null value 60% or more 
high_nan = ['icu_patients',
 'icu_patients_per_million',
 'hosp_patients',
 'hosp_patients_per_million',
 'weekly_icu_admissions',
 'weekly_icu_admissions_per_million',
 'weekly_hosp_admissions',
 'weekly_hosp_admissions_per_million',
 'total_vaccinations',
 'people_vaccinated',
 'people_fully_vaccinated',
 'new_vaccinations',
 'new_vaccinations_smoothed',
 'total_vaccinations_per_hundred',
 'people_vaccinated_per_hundred',
 'people_fully_vaccinated_per_hundred',
 'new_vaccinations_smoothed_per_million']

# columns to drop
droping = ['total_cases', 'new_cases', 'new_cases_smoothed', 
        'total_deaths', 'new_deaths', 'new_deaths_smoothed', 
        'total_cases_per_million', 'new_cases_per_million', 
        'new_cases_smoothed_per_million', 'total_deaths_per_million', 
        'new_deaths_per_million', 'new_deaths_smoothed_per_million']

# from section 1.6.1.
high_outliers = [
 'weekly_icu_admissions_per_million',
 'new_deaths_smoothed',
 'people_fully_vaccinated',
 'new_deaths_per_million',
 'new_tests_per_thousand',
 'hosp_patients',
 'population_density',
 'population',
 'new_cases',
 'icu_patients',
 'tests_per_case',
 'gdp_per_capita',
 'people_fully_vaccinated_per_hundred',
 'total_tests_per_thousand',
 'weekly_icu_admissions',
 'people_vaccinated',
 'new_tests',
 'weekly_hosp_admissions',
 'new_tests_smoothed_per_thousand',
 'new_cases_smoothed_per_million',
 'new_vaccinations_smoothed',
 'total_vaccinations_per_hundred',
 'new_vaccinations',
 'people_vaccinated_per_hundred',
 'new_deaths',
 'hospital_beds_per_thousand',
 'total_cases_per_million',
 'total_tests',
 'reproduction_rate',
 'positive_rate',
 'weekly_hosp_admissions_per_million']

In [ ]:
high_outliers2 = list(set(high_outliers) - set(droping) -set(high_nan))

## 4.1.  NaN Values

In [ ]:
# droping null values 
df3.dropna(subset=['continent'], inplace=True)

# replace missing values
df3.fillna(value=0, inplace=True)

## 4.2. Transformation

In [ ]:
# creating instance of labelencoder
le = LabelEncoder()

# label encoder
df3['iso_code'] = le.fit_transform(df3['iso_code'])
df3['continent'] = le.fit_transform(df3['continent'])
df3['year_month'] = le.fit_transform(df3['year_month'])

# transforming datetime into Gregorian calendar
df3['date'] = pd.to_datetime(df3['date'])
df3['date']=df3['date'].map(dt.datetime.toordinal)

## 4.3. Rescaling

In [ ]:
drop = ['total_deaths', 'total_cases', 'iso_code', 
        'location', 'date', 'tests_units', 'continent']

# robust scaler
rs = RobustScaler()

for i in high_outliers2:
    df3[i] = rs.fit_transform(df3[[i]].values)

In [ ]:
numerical = df3.select_dtypes(include='float64')

low_outliers = list(set(numerical.columns) - set(high_outliers2) - set(drop))

mms = MinMaxScaler()

for i in high_outliers:
    df3[i] = mms.fit_transform(df3[[i]].values)

# 5.0. Feature Selection

In [ ]:
# joining lists to drop
to_drop = list((set(droping) | (set(drop))))
to_drop = to_drop.copy()
to_drop.extend(high_nan)
to_drop

In [ ]:
# remove target variable
X_boruta = df3.drop(to_drop, axis=1)

# target variable
Y_boruta = df3[['total_cases']].copy()

In [ ]:
X_boruta.shape

In [ ]:
Y_boruta.shape

## 5.1. Boruta as Feature Selector

In [ ]:
# train random forest classifier

X_boruta_n = X_boruta.values

Y_boruta_n = Y_boruta.values

# define random forest regression
rf = RandomForestRegressor(n_jobs=-1)

# define Boruta feature selection method
feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=1)

# find all relevant features
feat_selector.fit(X_boruta_n, Y_boruta_n)

In [ ]:
## check selected features
cols_selected_boruta = feat_selector.support_.tolist()
columns_selected = df3.drop(to_drop, axis=1).loc[:, cols_selected_boruta].columns.tolist()

# list with features
print(len(columns_selected))
print(columns_selected)

* An area of acceptance: the features that are here are considered as predictive, so they are kept;
* An area of irresolution: Boruta is indecisive about the features that are in this area.

In [ ]:
# print results
acceptance = X_boruta.columns[feat_selector.support_].to_list()
irresolution = X_boruta.columns[feat_selector.support_weak_].to_list()
print('features in the acceptance:', acceptance)
print('features in the irresolution:', irresolution)

In [ ]:
# saving list
boruta_selection = ['total_tests', 'total_tests_per_thousand', 'new_tests_smoothed', 'positive_rate', 'tests_per_case', 'stringency_index', 'population', 'population_density', 'female_smokers', 'male_smokers', 'year_month']

## 5.2. Random Forest as Feature Selector

In [ ]:
# selecting Model and fit
model = RandomForestRegressor().fit(X_boruta, Y_boruta)
 
# plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_, index=X_boruta.columns)
feat_importances.nlargest(30).plot(kind='barh');

# 6.0. Split data into train and test

In [ ]:
# changing dtype of date

x = df3[boruta_selection]
y = df3.total_cases

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)

In [ ]:
x_train.shape, y_train.shape

In [ ]:
x_test.shape, y_test.shape

# 7.0. Machine Learning Modelling

## 7.1. Linear Regression

In [ ]:
# define model and fit
linear_r = LinearRegression().fit(x_train, y_train)

# predict
lr_pred = linear_r.predict(x_test)

# evaluate
lr_error = error('Linear Regression', y_test, lr_pred)

In [ ]:
# checking results
lr_error

## 7.2. K-Neighbors Regressor

In [ ]:
# define and fit model
kn = KNeighborsRegressor().fit(x_train, y_train)

# predict
kn_pred = kn.predict(x_test)

# evaluate
kn_error = error('KNeighbors Regressor', y_test, kn_pred)

In [ ]:
kn_error

## 7.3. Random Forest Regression

In [ ]:
# define and fit model 
rfr = RandomForestRegressor().fit(x_train, y_train)

# predict
rfr_pred = rfr.predict(x_test)

# evaluate
rfr_error = error('Random Forest Regression', y_test, rfr_pred)

In [ ]:
rfr_error

## 7.4. XGBoost Regressor

In [ ]:
# define and fit model
xgb = XGBRegressor().fit(x_train, y_train)

# predict
xgb_pred = xgb.predict(x_test)

# evaluate
xgb_error = error('XGBoost Regressor', y_test, xgb_pred)

In [ ]:
xgb_error

# 8.0. Cross Validation

## 8.1. Linear Regression CV

In [ ]:
# call function (see section 0.1)
linear_r_cv = cross_validation('Linear Regression CV', linear_r, x_train, y_train)
linear_r_cv

## 8.2. K-Neighbors Regressor CV

In [ ]:
# call function (see section 0.1)
kn_cv = cross_validation('K-Neighbors CV', kn, x_train, y_train)
kn_cv

## 8.3. Random Forest Regression CV

In [ ]:
rfr_cv = cross_validation('Random Forest CV', rfr, x_train, y_train)
rfr_cv

## 8.4. XGBoost Regressor CV

In [ ]:
xgb_cv = cross_validation('XGBoost CV', xgb, x_train, y_train)
xgb_cv

# 9.0. Compare Model's Performance

## 9.1. Single Performance

In [ ]:
model_sp = pd.concat([lr_error, kn_error, rfr_error, xgb_error])
model_sp.sort_values('RSME').reset_index(drop=True)

## 9.2. Real Performance - Cross Validation

In [ ]:
model_rp = pd.concat([linear_r_cv, kn_cv, rfr_cv,  xgb_cv])
model_rp.sort_values('RSME').reset_index(drop=True)

# 10.0. Tuning Best Model

Since XGBooster is the second best model (in terms of errors) and it took less time to run, I'll follow this cycle using it.

## 10.1. Tuning Model - GridSearchCV

In [ ]:
# params
param_grid = {
    'n_estimators' : [100, 200, 500],
    'objective': ['reg:squarederror'],
    'eta': [0.1],
    "max_depth": [9],
    "gamma": [0],
    "reg_lambda": [10],
    "scale_pos_weight": [1],
    "subsample": [1],  # Fix subsample
    "colsample_bytree": [0.7, 0.8],  # Fix colsample_bytree
}
xgb_t = XGBRegressor()            
xgb_grid = HalvingGridSearchCV(xgb_t,
                        param_grid, cv=5, n_jobs=-1,
                        verbose=True).fit(x_train, y_train)

In [ ]:
xgb_grid.best_estimator_

In [ ]:
# xgboost with best params
xgb_tun = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, eta=0.1, gamma=0,
             gpu_id=-1, importance_type='gain', interaction_constraints='',
             learning_rate=0.100000001, max_delta_step=0, max_depth=9,
             min_child_weight=1, monotone_constraints='()',
             n_estimators=500, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=10, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)\
                 .fit(x_train, y_train)

# predict
xgb_t_pred = xgb_tun.predict(x_test)

# evaluate
xgb_tuning_error = error('XGBoost Regressor +', y_test, xgb_t_pred)

In [ ]:
xgb_tuning_error

## 10.2. XGBoost Regressor + CV

In [ ]:
xgb_t_cv = cross_validation('XGBoost + CV', xgb_tun, x_train, y_train)
xgb_t_cv

# 11.0. Final Compare Model's Performance

## 11.1. Single Performance

In [ ]:
model_sp = pd.concat([lr_error, kn_error, rfr_error, xgb_error, xgb_tuning_error])
model_sp.sort_values('MAE').reset_index(drop=True)

## 11.2. Real Performance - Cross Validation

In [ ]:
model_rp = pd.concat([linear_r_cv, kn_cv, rfr_cv, xgb_cv, xgb_t_cv])
model_rp.sort_values('MAE').reset_index(drop=True)

In [ ]:
# saving models

pickle.dump(xgb_tun, open('model/xgb_tuned_c2.pkl', 'wb'))
pickle.dump(xgb, open('model/xgb_c2.pkl', 'wb'))

# 12.0. Model Performance

In [ ]:
# loading models
xgb_tun = pickle.load(open('model/xgb_tuned.pkl', 'rb'))

# selecting columns
columns_name = ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10'] 
columns_name2 = boruta_selection

# zip columns
d_columns = dict(zip(columns_name2, columns_name))

# changing columns name
df4 = df3.copy()
df4 = df3.rename(columns = d_columns)

# target variable
df4['total_cases'] = df3.total_cases

# prediction columns
df4['total_cases_pred'] = xgb_tun.predict(df4[columns_name])

## 12.1. Error and Error Rate

In [ ]:
# get error
df4['error'] = df4['total_cases'] - df4['total_cases_pred'] 

# get error rate
df4['error_rate'] = df4['total_cases'] / df4['total_cases_pred']

## 12.2. Ploting Results

In [ ]:
# prepare dataset for plot
df5 = df3.copy()

# include columns
df5['total_cases_pred'] = df4['total_cases_pred']
df5['error'] = df4['error']
df5['error_rate'] = df4['error_rate']

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2)

# plot prediction x target
sns.lineplot(data=df5, x='year_month', y='total_cases', label='Total Cases',\
     color='red', ax=ax[0,0])
sns.lineplot(data=df5, x='year_month', y='total_cases_pred', label='Prediction', ax=ax[0,0])
ax[0,0].legend()
ax[0,0].set_title('PREDICTION PERFORMANCE')

# plot error
sns.lineplot(data=df5, x='year_month', y='error_rate', label='Error Rate', ax=ax[1,0])
ax[1,0].axhline(1, linestyle='--')
ax[1,0].legend()
ax[1,0].set_title('ERROR RATE')

# hist error
sns.histplot(df5.error, kde=True, ax=ax[0,1])
ax[0,1].set_title('ERROR')

# plot prediction x error
sns.scatterplot(data=df5, x='total_cases_pred', y='error', ax=ax[1,1])
ax[1,1].set_title('PREDIC X ERROR')

# save plot
plt.savefig('img/fig17')

# plot
plt.show()

In [ ]:
# check gaussian
stats.probplot(df4.error, plot=plt)

# save plot
plt.savefig('img/fig18')

# plot
plt.show()